# Exemple de mise en forme des données

In [1]:
import sys
sys.path.append('/users/loulou/cedia/OLG_CAN/demo/simgen/')
import warnings
warnings.filterwarnings("ignore")

On utilise trois fonctions de compas: 
* bdsps: manipule la base BDSPS de Statistiques Canada pour mettre en forme certaines variables et créer les registres avec NAS de dominants. 
* parse: reformule les noms de variables à l'aide de dictionnaires
* population: crée la structure de population

In [2]:
from simgen import bdsps, population, parse

## Nettoyage de la BDSPS

Compas vient avec une fonction, bdsps qui nettoie les données de la BDSPS, crée les NAS et les trois registres. Cette fonction peut être adapté pour d'autres bases de données. Fait important à noter, la fonction BDSPS calibre aussi les poids des répondants, par âge et sexe pour s'arrimer sur la population québécoise de 2017, selon l'ISQ. 

In [3]:
hh,sp,kd = bdsps('../raw/bdsps2017.dta')

Voici ce à quoi ressemble les registres des dominants, conjoints et enfants avant la mise en forme finale. 

In [4]:
hh.head()

,hhid,age,byr,male,immig,newimm,yrsimm,educ4,inschool,married,spflag,wgt,pn,nkids
nas,,,,,,,,,,,,,,
0,0,33,1984,True,False,False,NaN,none,False,False,0,397.943,0,2.0
1,0,13,2004,False,False,False,NaN,none,True,False,0,414.793,2,0.0
2,0,15,2002,True,False,False,NaN,none,True,False,0,450.728,2,0.0
3,1,33,1984,True,False,False,NaN,none,False,False,0,398.808,0,2.0
4,1,13,2004,False,False,False,NaN,none,True,False,0,415.695,2,0.0


In [5]:
sp.head()

,hhid,age,byr,male,immig,newimm,yrsimm,educ4,inschool,married,spflag,wgt,pn
nas,,,,,,,,,,,,,
21,7,39,1978,True,False,False,NaN,des,False,True,1,26.9994,1
22,7,47,1970,False,False,False,NaN,none,False,True,1,24.59,0
26,8,39,1978,True,False,False,NaN,des,False,True,1,26.9994,1
27,8,47,1970,False,False,False,NaN,none,False,True,1,24.59,0
31,9,39,1978,True,False,False,NaN,des,False,True,1,26.9994,1


In [6]:
kd.head()

,hhid,age,byr,male,immig,newimm,yrsimm,educ4,inschool,married,spflag,wgt,pn,nparents
nas,,,,,,,,,,,,,,
0,0,13,2004,False,False,False,NaN,none,True,False,0,414.793,2,1
0,0,15,2002,True,False,False,NaN,none,True,False,0,450.728,2,1
3,1,13,2004,False,False,False,NaN,none,True,False,0,415.695,2,1
3,1,15,2002,True,False,False,NaN,none,True,False,0,451.708,2,1
6,2,13,2004,False,False,False,NaN,none,True,False,0,415.695,2,1


On partitionne pour obtenir seulement les nouveaux immigrants dans des registres différents. 

In [7]:
imm = hh[hh.newimm]
imm_nas = imm.index
sp_imm = sp.loc[sp.index.isin(imm_nas),:]
kd_imm = kd.loc[kd.index.isin(imm_nas),:]

## Adapter les noms de variables pour Compas

Une étape importante est de faire la correspondance entre les noms de variables des registres de la BDSPS et ceux dans COMPAS. Pour ce faire, nous avons la classe parse(). Cette classe permet, à l'aide d'un dictionnaire de faire cette correspondance, pour chaque registre. Par défaut, parse() suppose les mêmes noms de variables que dans Compas.  

In [8]:
parsing = parse()
parsing.map_hh

{'wgt': 'wgt',
 'byr': 'byr',
 'male': 'male',
 'educ': 'educ',
 'insch': 'insch',
 'nkids': 'nkids',
 'married': 'married'}

On fait ensuite notre correspondance propre à la BDSPS. 

In [9]:
parsing.map_hh['educ'] = 'educ4'
parsing.map_hh['insch'] = 'inschool'
parsing.map_sp['educ'] = 'educ4'
parsing.map_sp['insch'] = 'inschool'
parsing.map_kd['insch'] = 'inschool'

In [10]:
parsing.map_hh

{'wgt': 'wgt',
 'byr': 'byr',
 'male': 'male',
 'educ': 'educ4',
 'insch': 'inschool',
 'nkids': 'nkids',
 'married': 'married'}

On ajuste ensuite à l'aide de ces dictionnaires les trois registres. 

In [11]:
hh = parsing.dominants(hh)
sp = parsing.spouses(sp)
kd = parsing.kids(kd)

In [12]:
hh.head()

,wgt,byr,male,educ,insch,nkids,married
nas,,,,,,,
0,397.942563,1984,True,none,False,2,False
1,414.793316,2004,False,none,True,0,False
2,450.727786,2002,True,none,True,0,False
3,398.807655,1984,True,none,False,2,False
4,415.695040,2004,False,none,True,0,False


In [13]:
imm = parsing.dominants(imm)
sp_imm = parsing.spouses(sp_imm)
kd_imm = parsing.kids(kd_imm)

## Création des structures de population et sauvegarde des données

Après avoir ajusté les registres, on peut les introduire dans des structures de population. La classe population permet de faire ceci. On peut ensuite sauvegarder les objets en format pickle en donnant le nom du fichier. 

In [14]:
pop = population()
pop.input(hh,sp,kd)
pop.save('startpop')

In [15]:
newimm = population()
newimm.input(imm,sp_imm,kd_imm)
newimm.save('newimmpop')